In [1]:
import torch
from reconstruction import AE
from datasets import MeshData
from utils import utils, DataLoader, mesh_sampling, sap
import numpy as np
import pyvista as pv
from skimage import measure
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
from IPython.display import display
import meshplot as mp
import os, sys
from math import ceil
from scipy.ndimage import zoom
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Meshplot left an annoying print statement in their code. Using this context manager to supress it...
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [3]:
device = torch.device('cuda', 0)
# Set the path to the saved model directory
model_path = "/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA/raw/torus_two/models/26"

# Load the saved model
model_state_dict = torch.load(f"{model_path}/model_state_dict.pt")
in_channels = torch.load(f"{model_path}/in_channels.pt")
out_channels = torch.load(f"{model_path}/out_channels.pt")
latent_channels = torch.load(f"{model_path}/latent_channels.pt")
spiral_indices_list = torch.load(f"{model_path}/spiral_indices_list.pt")
up_transform_list = torch.load(f"{model_path}/up_transform_list.pt")
down_transform_list = torch.load(f"{model_path}/down_transform_list.pt")
std = torch.load(f"{model_path}/std.pt")
mean = torch.load(f"{model_path}/mean.pt")
template_face = torch.load(f"{model_path}/faces.pt")

# Create an instance of the model
model = AE(in_channels, out_channels, latent_channels,
           spiral_indices_list, down_transform_list,
           up_transform_list)
model.load_state_dict(model_state_dict)
model.to(device)
# Set the model to evaluation mode
model.eval()

AE(
  (en_layers): ModuleList(
    (0): SpiralEnblock(
      (conv): SpiralConv(3, 8, seq_length=9)
    )
    (1-2): 2 x SpiralEnblock(
      (conv): SpiralConv(8, 8, seq_length=9)
    )
    (3): SpiralEnblock(
      (conv): SpiralConv(8, 16, seq_length=9)
    )
    (4): Linear(in_features=1568, out_features=32, bias=True)
  )
  (de_layers): ModuleList(
    (0): Linear(in_features=16, out_features=1568, bias=True)
    (1): SpiralDeblock(
      (conv): SpiralConv(16, 16, seq_length=9)
    )
    (2): SpiralDeblock(
      (conv): SpiralConv(16, 8, seq_length=9)
    )
    (3-4): 2 x SpiralDeblock(
      (conv): SpiralConv(8, 8, seq_length=9)
    )
    (5): SpiralConv(8, 3, seq_length=9)
  )
  (reg_sq): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Linear(in_features=8, out_features=8, bias=True)
    (4): BatchNorm1d(8,

In [4]:
z = torch.zeros(16)
plot=None
@mp.interact(**{f'z[{i}]': FloatSlider(min=-1.0, max=1.0, step=0.2, value=0) for i in range(16)})
def show(**kwargs):
    global plot
    global z
    z = torch.tensor([kwargs[f'z[{i}]'] for i in range(16)])
    with torch.no_grad():
        z = z.to(device)
        #print(z)
        pred = model.decoder(z)

        reshaped_pred = (pred.view(-1, 3).cpu() * std) + mean
        reshaped_pred = reshaped_pred.cpu().numpy()
        print(reshaped_pred.shape)

    verts = reshaped_pred
    pcd = o3d.io.read_triangle_mesh('/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA/template/template.ply')
    faces = np.asarray(pcd.triangles)

    if plot is None:
        plot = mp.plot(verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=verts, faces=faces)
        display(plot._renderer)

interactive(children=(FloatSlider(value=0.0, description='z[0]', max=1.0, min=-1.0, step=0.2), FloatSlider(val…

In [5]:
latent_channels = torch.load(f"{model_path}/latent_channels.pt")
angles = torch.load(f"{model_path}/angles.pt")

FileNotFoundError: [Errno 2] No such file or directory: '/home/jakaria/Explaining_Shape_Variability/src/DeepLearning/compute_canada/guided_vae/data/CoMA/raw/torus_two/models/195/angles.pt'

In [ ]:
angles


tensor([[ 0.8317],
        [-0.4349],
        [-0.2465],
        [-0.6473],
        [-0.6754],
        [-0.0421],
        [ 0.0180],
        [-0.2866],
        [ 0.2625],
        [ 0.2425],
        [-0.9760],
        [ 0.1503],
        [-0.9840],
        [-0.9960],
        [-0.1944],
        [-0.4549],
        [-0.1663],
        [-0.4749],
        [ 0.5591],
        [-0.9198],
        [ 0.5711],
        [ 0.9599],
        [ 0.8998],
        [ 0.3106],
        [ 0.9158],
        [ 0.6593],
        [-0.0862],
        [-0.6914],
        [-0.3387],
        [ 0.9559],
        [ 0.1743],
        [-0.5150],
        [ 0.1623],
        [ 0.6473],
        [ 0.1984],
        [-0.7315],
        [-0.9559],
        [-0.7876],
        [-0.4509],
        [-0.9158],
        [-0.9359],
        [-0.4269],
        [-0.7515],
        [ 0.4629],
        [ 0.1182],
        [-0.9078],
        [-0.5992],
        [ 0.6032],
        [-0.4950],
        [ 0.4709]], device='cuda:0')